#### Lab2 - Data Collection and Pre-processing - Manu Mathew - 8990691

##### Adding the required imports 

In [41]:
import pandas as pd
from dataclasses import dataclass

##### Load raw CSV, display first 3 rows

In [42]:
df = pd.read_csv(dataset_path)
print(df.head(3))

         date  customer_id   product  product_id  price  quantity  \
0  2025-05-26         3000    Apples           1  13.23         3   
1  2025-05-04         3001      Milk           4  27.94         1   
2  2025-05-19         3002  Tomatoes           9  13.72         6   

  shipping_city  coupon_id  
0        Ottawa          1  
1     Vancouver          4  
2        Ottawa          5  


##### Justify dict vs namedtuple vs class
- Dictionary: Mutable, unordered collection of key-value pairs.
- NamedTuple: Similar to tuple, but it is object with named fields.
- Class: A blueprint for creating objects with attributes and behaviors.

I have chosen class, so that I can include the implementations of ingest, wrangle, clean, transform, feature-engineer and serialization methods, so that the codebase is more scalable.

##### Implement Transaction class and use it to populate data structure an object-oriented data structure

In [43]:
@dataclass
class Transaction:
    date: str
    customer_id: str
    product:str
    product_id: str
    price: float
    quantity: int
    shipping_city: str
    coupon_id:int


#### Create a method load_transactions() which takes the dataset path and returns list of Transactions

In [44]:
def load_transactions(path: str) -> list[Transaction]:
    df = pd.read_csv(path)
    transactions = []

    for record in df.to_dict(orient="records"):
        transaction = Transaction(
            date=record["date"],
            customer_id=record["customer_id"],
            product = record["product"],
            product_id=record["product_id"],
            price=record["price"],
            quantity=record["quantity"],
            coupon_id=record["coupon_id"],
            shipping_city=record["shipping_city"]
        )
        transactions.append(transaction)
    print(transactions)
    return transactions




#### Main method

In [45]:
if __name__ == "__main__":
    dataset_path = "data/Grocery_Transactions.csv";
    listOfTransactions = load_transactions(dataset_path);
    print(listOfTransactions)

[Transaction(date='2025-05-26', customer_id=3000, product='Apples', product_id=1, price=13.23, quantity=3, shipping_city='Ottawa', coupon_id=1), Transaction(date='2025-05-04', customer_id=3001, product='Milk', product_id=4, price=27.94, quantity=1, shipping_city='Vancouver', coupon_id=4), Transaction(date='2025-05-19', customer_id=3002, product='Tomatoes', product_id=9, price=13.72, quantity=6, shipping_city='Ottawa', coupon_id=5), Transaction(date='2025-05-16', customer_id=3003, product='Tomatoes', product_id=9, price=44.79, quantity=4, shipping_city='Halifax', coupon_id=2), Transaction(date='2025-05-07', customer_id=3004, product='Tomatoes', product_id=9, price=2.21, quantity=1, shipping_city='Winnipeg', coupon_id=3), Transaction(date='2025-05-04', customer_id=3005, product='Orange Juice', product_id=14, price=29.56, quantity=10, shipping_city='Calgary', coupon_id=2), Transaction(date='2025-05-12', customer_id=3006, product='Bananas', product_id=2, price=3.39, quantity=8, shipping_ci